In [1]:
import re
def img_url_ct(body):
    ct = 0
    pattern = r'<a(.*?)>'
    matches = re.findall(pattern, body, re.DOTALL)
    ct+=len(matches)
    pattern = r'<svg(.*?)>'
    matches = re.findall(pattern, body, re.DOTALL)
    ct+=len(matches)
    pattern = r'<img(.*?)>'
    matches = re.findall(pattern, body, re.DOTALL)
    ct+=len(matches)
    return ct
img_url_ct('''<p>I am new to Kotlin. I have converted some code from Java, but it seems like there's something wrong. The R in findViewById(R.id.my_id) is highlighted in red and it shows this message:</p>
<blockquote>
<p>Unresolved reference: R</p>
</blockquote>
<p>I've been looking for a solution, but I seem not to figure it out. So what should I do?</p>
<p>Here's a screenshot:</p>
<p><a href="https://i.stack.imgur.com/n4TCM.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/n4TCM.png" alt="Enter image description here" /></a></p>
''')

2

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

def get_code(text):
        pattern = r'<code>(.*?)</code>'
#         print(text)
# Find all matches
        matches = re.findall(pattern, text, re.DOTALL)
        text_without_code = re.sub(pattern, '', text, flags=re.DOTALL)
#         print(matches[0])
        pattern4 = r'<.*?>'

# Remove all HTML tags
        text_without_code = re.sub(pattern4, '', text_without_code)
        cleaned_text = re.sub(r'\s+', ' ', text_without_code.strip())

        pattern2 = r'\n'

# Find all matches
        matches2 = re.findall(pattern2, text)
        str1 = ' '.join(matches)
        cleaned_str1 = re.sub(r'\s+', ' ', text_without_code.strip())
        matches3 = re.findall(pattern2,str1)
        
        return len(matches2), len(matches3)-len(matches)

In [3]:

def accept_rate_to_categorical(val):
    if str(val)=='nan':
        return 0
    elif val < 33:
        return 1
    elif val < 66:
        return 2
    return 3
def interval(val):
    if str(val)=='nan' or val <0:
        return -1
    return val
ids = 53545800
# ids = question id
Df = {}
# Body = request_data['body']
Body = r'''<p>I'm getting the following error in the console:
<code>Error : Cannot find module</code>.</p>
<p>Here is the full error. What should I do?</p>
<pre><code>internal/modules/cjs/loader.js:582
    throw err;
    ^

Error: Cannot find module 'C:\Users\User\Desktop\NodeJsProject\app.js'
    at Function.Module._resolveFilename (internal/modules/cjs/loader.js:580:15)
    at Function.Module._load (internal/modules/cjs/loader.js:506:25)
    at Function.Module.runMain (internal/modules/cjs/loader.js:741:12)
    at startup (internal/bootstrap/node.js:285:19)
    at bootstrapNodeJSCore (internal/bootstrap/node.js:739:3)
</code></pre>'''
Df['LOC'], Df['Question_Length'] = get_code(Body)
Df['Total count urls and imgs'] =  img_url_ct(Body)
url = f"https://api.stackexchange.com/2.3/questions/{ids}"
params = {
                "site": "stackoverflow"
            }
response = requests.get(url, params=params)

if response.status_code == 200:
    ans = response.json()['items'][0] 
    Df['View Count'] = ans["view_count"]
    Df['Answer Count'] = ans["answer_count"]
    Df['Score'] = ans["score"]
    try:
        Owner = ans["owner"]["account_id"]
        url = f"https://api.stackexchange.com/2.3/users/{Owner}"
        params = {
            "site": "stackoverflow"
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            # print(response.json())
            ans2 = response.json()['items'][0] # Assuming the response will contain only one item
            Df['User_Reputation'] = ans2['reputation']
            Df['Bronze_badge'] = ans2["badge_counts"]["bronze"]
            Df['Gold_badge'] = ans2["badge_counts"]["gold"]
            Df['Silver_badge'] = ans2["badge_counts"]["bronze"]
            Df['cAccept_rate'] = accept_rate_to_categorical(ans2["accept_rate"])
        else:
            Df['User_Reputation'] = 0
            Df['Bronze_badge'] = 0
            Df['Gold_badge'] = 0
            Df['Silver_badge'] = 0
            Df['cAccept_rate'] = 0
    except:
        Df['User_Reputation'] = 0
        Df['Bronze_badge'] = 0
        Df['Gold_badge'] = 0
        Df['Silver_badge'] = 0
        Df['cAccept_rate'] = 0


    url = f"https://api.stackexchange.com/2.3/questions/{ids}/answers"
    params = {
        "site": "stackoverflow",
        "sort": "asc",
        "order":"creation"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        try:
            ans3 = response.json()['items'][0]
            Df['Interval from first'] = ans3["creation_date"] - ans["creation_date"]
        except:
            Df['Interval from first'] = -1
    else:
        Df['Interval from first'] = -1
    try:
        AccpId = ans["accepted_answer_id"]
        url = f"https://api.stackexchange.com/2.3/answers/{ids}"
        params = {
            "site": "stackoverflow"
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            ans4 = response.json()['items'][0]
            Df['cInterval from accepted'] = ans4["creation_date"] - ans["creation_date"]
        else : Df['cInterval from accepted'] = -1
    except:
        Df['cInterval from accepted'] = -1